In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score, LeaveOneOut
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

In [2]:
from abc import ABC, abstractmethod

class KNearestNeighbors(ABC):
    ''' Base class for implementation of interface of KNN model '''
    
    def __init__(self, n_neighbors=1):
        self.n_neighbors = n_neighbors
        self.X_train = None
        self.y_train = None

    @abstractmethod
    def predict(self, X_test):
        pass
    
    def set_params(self, n_neighbors=None):
        if n_neighbors:
            self.n_neighbors = n_neighbors
    
    def __find_neighbors(self, x_test):
        if self.X_train is None:
            raise Exception('Training data has not been loaded')
            
        if x_test is None:
            raise Exception('Test data has not been provided')
            
        distances = list()
        
        for i, x_train in enumerate(self.X_train):
            distance = np.linalg.norm(x_train - x_test)
            distances.append((i, distance))
        
        distances.sort(key=lambda x: x[1])

        return [distance[0] for distance in distances][:self.n_neighbors]
    
    def retrieve_neighbors(self, X_test):
        if X_test is None:
            raise Exception('Test data has not been provided')
        
        return self.__find_neighbors(X_test)
    
    def fit(self, X_train, y_train):
        if X_train is None:
            raise Exception('Training feature data has not bee provided')
        
        if y_train is None:
            raise Exception('Training prediction data has not been provided')
            
        self.X_train = X_train
        self.y_train = y_train

In [3]:
class KNearestNeighborsRegressor(KNearestNeighbors):
    ''' Implementation for regression tasks using KNN algorithm '''
    
    def __init__(self, n_neighbors=1):
        super().__init__(n_neighbors)

    def predict(self, X_test):
        for x_test in X_test:
            print('Test entry:', x_test)
            for index in self.retrieve_neighbors(x_test):
                print('Neighbor:', self.X_train[index], self.y_train[index])
        return [
            np.average([self.y_train[index] for index in self.retrieve_neighbors(x_test)]) for x_test in X_test
        ]

In [4]:
X_train = np.array([
    [1, 7],
    [2, 5],
    [2, 6],
    [3, 3],
    [7, 1],
    [3, 1],
    [4, 2],
    [5, 4],
    [5, 6]
])

y_train = np.array([
    25,
    21,
    14,
    32,
    14,
    14,
    25,
    18,
    12
])

k = 6

In [5]:
model = KNearestNeighborsRegressor(n_neighbors=k)
model.fit(X_train, y_train)
y_pred = model.predict(X_train)

neigh = KNeighborsRegressor(n_neighbors=k)
neigh.fit(X_train, y_train)

print('Implementation:', y_pred)
print('sklearn:', neigh.predict(X_train))

Test entry: [1 7]
Neighbor: [1 7] 25
Neighbor: [2 6] 14
Neighbor: [2 5] 21
Neighbor: [5 6] 12
Neighbor: [3 3] 32
Neighbor: [5 4] 18
Test entry: [2 5]
Neighbor: [2 5] 21
Neighbor: [2 6] 14
Neighbor: [1 7] 25
Neighbor: [3 3] 32
Neighbor: [5 4] 18
Neighbor: [5 6] 12
Test entry: [2 6]
Neighbor: [2 6] 14
Neighbor: [2 5] 21
Neighbor: [1 7] 25
Neighbor: [5 6] 12
Neighbor: [3 3] 32
Neighbor: [5 4] 18
Test entry: [3 3]
Neighbor: [3 3] 32
Neighbor: [4 2] 25
Neighbor: [3 1] 14
Neighbor: [2 5] 21
Neighbor: [5 4] 18
Neighbor: [2 6] 14
Test entry: [7 1]
Neighbor: [7 1] 14
Neighbor: [4 2] 25
Neighbor: [5 4] 18
Neighbor: [3 1] 14
Neighbor: [3 3] 32
Neighbor: [5 6] 12
Test entry: [3 1]
Neighbor: [3 1] 14
Neighbor: [4 2] 25
Neighbor: [3 3] 32
Neighbor: [5 4] 18
Neighbor: [7 1] 14
Neighbor: [2 5] 21
Test entry: [4 2]
Neighbor: [4 2] 25
Neighbor: [3 3] 32
Neighbor: [3 1] 14
Neighbor: [5 4] 18
Neighbor: [7 1] 14
Neighbor: [2 5] 21
Test entry: [5 4]
Neighbor: [5 4] 18
Neighbor: [5 6] 12
Neighbor: [3 3] 32
N

In [6]:
print('Training error using kNN regressor with k = 6:', np.power(y_pred - y_train, 2).mean())

Training error using kNN regressor with k = 6: 39.50925925925925


In [7]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)

print(model.intercept_, model.coef_)
y_pred = model.predict(X_train)
print('Training error using linear model:', np.power(y_pred - y_train, 2).mean())

30.131075914800654 [-2.03850355 -0.88421628]
Training error using linear model: 31.71864190788276


In [12]:
X = np.array([
    [-2],
    [2],
    [1],
    [-1],
    [0],
    [3]
])

y = np.array([
    4,
    4,
    1,
    1,
    0,
    9
])

k = 2
K = 3

In [14]:
model = KNeighborsRegressor(n_neighbors=k)
kf = KFold(n_splits=K, shuffle=False)

for train, test in kf.split(X):
    print('TRAIN:', train)
    print('TEST:', test)
    model.fit(X[train], y[train])
    y_pred = model.predict(X[test])
    print('True:', X[test], y[test])
    print('Predicted:', y_pred)
    print(mean_squared_error(y[test], y_pred))

cv_score = -cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')

print(cv_score)
print(cv_score.mean())

TRAIN: [2 3 4 5]
TEST: [0 1]
True: [[-2]
 [ 2]] [4 4]
Predicted: [0.5 5. ]
6.625
TRAIN: [0 1 4 5]
TEST: [2 3]
True: [[ 1]
 [-1]] [1 1]
Predicted: [2. 2.]
1.0
TRAIN: [0 1 2 3]
TEST: [4 5]
True: [[0]
 [3]] [0 9]
Predicted: [1.  2.5]
21.625
[ 6.625  1.    21.625]
9.75


In [28]:
model = linear_model.LinearRegression()
model.fit(X, y)

loo = LeaveOneOut()

for train, test in loo.split(X):
    print('========================')
    print('TRAIN:', train)
    print('TEST:', test)
    X_train_g = np.take(X, train).reshape(-1, 1)
    y_train_g = np.take(y_train, train).reshape(-1, 1)
    X_test_g = np.take(X_train, test).reshape(-1, 1)
    y_test_g = np.take(y_train, test).reshape(-1, 1)
    print('Train')
    print(np.append(X_train_g, y_train_g, axis=1))
    
    m = linear_model.LinearRegression()
    m.fit(X_train_g, y_train_g)
    print('Intercept and coef:', m.intercept_, m.coef_)
    print('True:', y_test_g)
    print('Predicted:', m.predict(X_test_g))
    
cv_score = -cross_val_score(m, X_train, y_train, cv=loo, scoring='neg_mean_squared_error', verbose=4)
       
print(cv_score)
print(cv_score.mean())

TRAIN: [1 2 3 4 5]
TEST: [0]
Train
[[ 2  4]
 [ 1  1]
 [-1  1]
 [ 0  0]
 [ 3  9]]
Intercept and coef: [1.] [[2.]]
True: [[4]]
Predicted: [[-3.]]
TRAIN: [0 2 3 4 5]
TEST: [1]
Train
[[-2  4]
 [ 1  1]
 [-1  1]
 [ 0  0]
 [ 3  9]]
Intercept and coef: [2.78378378] [[1.08108108]]
True: [[4]]
Predicted: [[4.94594595]]
TRAIN: [0 1 3 4 5]
TEST: [2]
Train
[[-2  4]
 [ 2  4]
 [-1  1]
 [ 0  0]
 [ 3  9]]
Intercept and coef: [3.1627907] [[1.09302326]]
True: [[1]]
Predicted: [[4.25581395]]
TRAIN: [0 1 2 4 5]
TEST: [3]
Train
[[-2  4]
 [ 2  4]
 [ 1  1]
 [ 0  0]
 [ 3  9]]
Intercept and coef: [2.86486486] [[0.91891892]]
True: [[1]]
Predicted: [[1.94594595]]
TRAIN: [0 1 2 3 5]
TEST: [4]
Train
[[-2  4]
 [ 2  4]
 [ 1  1]
 [-1  1]
 [ 3  9]]
Intercept and coef: [3.25581395] [[0.90697674]]
True: [[0]]
Predicted: [[3.25581395]]
TRAIN: [0 1 2 3 4]
TEST: [5]
Train
[[-2  4]
 [ 2  4]
 [ 1  1]
 [-1  1]
 [ 0  0]]
Intercept and coef: [2.] [[-1.40433339e-16]]
True: [[9]]
Predicted: [[2.]]
[CV] END ........................

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
